In [164]:
import requests

In [165]:
jsn = requests.get('http://7bfeba68.ngrok.io/transactions').json()

In [166]:
import pandas as pd

In [167]:
df = pd.DataFrame(jsn)

In [168]:
places = df.counterPartyName.unique()

In [192]:
df.sort_values(by='bookingDate').usage.unique()

array(['ÜBERWEISUNG  Miete', 'POS MIT PIN. Berlin Mitte',
       'SEPA-BASISLASTSCHRIFT SVWZ+ TK-BuchNr 33306543912 Monat 03/16 K258289245 Beitraege ',
       'SEPA-BASISLASTSCHRIFT Abrechnungfuer 0178-6571111RNGNR 1163365100',
       'POS MIT PIN. Einkauf', 'Rech 69788786',
       'ÜBERWEISUNG Gehalt Februar', 'Barauszahlung, Schule',
       'SEPA-BASISLASTSCHRIFT Abrechnungfuer 0178-6571111RNGNR 1163365110',
       'SEPA-BASISLASTSCHRIFT SVWZ+ TK-BuchNr 33306543912 Monat 04/16 K258289245 Beitraege',
       'Ihr Drogeriemarkt am Platz', 'ÜBERWEISUNG Gehalt März',
       'SEPA-BASISLASTSCHRIFT SVWZ+ TK-BuchNr 33306543912 Monat 05/16 K258289245 Beitraege',
       'SEPA-BASISLASTSCHRIFT Abrechnungfuer 0178-6571111RNGNR 1163365120',
       'Barauszahlung, Markt', 'ÜBERWEISUNG Gehalt April',
       'Joga fuer Anfaenger', 'Sport', 'Rechnung',
       'SEPA-BASISLASTSCHRIFT Abrechnungfuer 0178-6571111RNGNR 1163365130',
       'SEPA-BASISLASTSCHRIFT SVWZ+ TK-BuchNr 33306543912 Monat 06/16 K258

In [170]:
places

array(['BlaBlaCar', 'Studentenwerk Berlin', 'Rewe AG', 'Telekom AG',
       'Techniker Krankenkasse', 'E-PLUS SERVICE GMBH', 'Deutsche Bahn AG',
       "Real London Souvenier's LHR", 'Lufthansa AG',
       'To the golden elephant', 'Eastlondon Train association',
       'Claudia Müller', 'Airbnb', 'Barclays', 'Icebar London',
       'London Eye', 'London Duck Tours',
       'The one and only Steak in London', 'Hollister Regent Street',
       'Abercrombie & Fitch Mayfair',
       "Victoria's Secret Brookstreet, London", 'Waitrose', 'Starbucks',
       'Madame Tussauds London', "Domino's Pizza",
       'London Oyster Tavelcard', 'Marktkauf', 'Aldi Nord',
       'Annemarie Theke', 'Amazon S.a.r.L.', 'Kölner Dom', 'UBER',
       'Kaffe Brandenburg', 'Rheintours', 'McDonalds',
       'Kölner Schokoladenmuseum', 'Studentenwerk Berlin, Sport',
       'Deutsche Bank', 'DM', 'Blablacar', 'Alte Pinakothek', 'Gamsbar',
       'P1', "L'Osteria am Gasteig", 'Vapiano Odeonsplatz', 'IKEA',
       'K

In [183]:
used = [' BlaBlaCar ', ' Uber ', 'museum', ' Airbnb ', 'hotel', 'bar', ' McDonalds ', ' Lyft ', 'taxi']

In [184]:
from geotext import GeoText
cities_list = set()
for place in places:
    for tok in place.split():
        cities = GeoText(tok)
        cities_list = cities_list.union(set(cities.cities))
print(cities_list)

{'Berlin', 'Hollister', 'London', 'Tours', 'Victoria'}


In [185]:
import re

In [186]:
for patt in used:
    if re.findall('{}'.format(patt.lower()), ' '.join(places).lower()):
        print(patt.strip())

BlaBlaCar
Uber
museum
Airbnb
bar
McDonalds


In [ ]:
from datetime import datetime

In [ ]:
d = datetime.strptime(flights[1]['InboundLeg']['DepartureDate'], '%Y-%m-%dT%H:%M:%S')

In [1]:
import time
from datetime import datetime, timedelta

import requests

from skyscanner_api_ import DataProvider
import pandas as pd

API_KEY = 'ma595491219569679758263226220714'

In [2]:
from skyscanner_live_pricing import LivePricing

In [86]:
outbound_date = (datetime.today() + timedelta(days=2)).date()
inbound_date = (datetime.today() + timedelta(days=7)).date()

In [87]:
cheapest = LivePricing(
        DataProvider.get_suggestions('Warszawa')[1]['code'].split('-')[0],
        DataProvider.get_suggestions('Zurych')[-1]['code'].split('-')[0],
        outbound_date,
        inbound_date,
        1).get_cheapest()

In [88]:
cheapest.keys()

dict_keys(['Segments', 'Status', 'Legs', 'Query', 'Itineraries', 'Currencies', 'Carriers', 'SessionKey', 'Places', 'Agents'])

In [131]:
from copy import deepcopy

def del_keys(d):
    copy = deepcopy(d)
    for key in d.keys():
        if key not in {'Carriers', 'arrival_date', 'arrival_time', 'time', 'date', 'Duration'}:
            del copy[key]
    return copy

In [137]:
carriers_mapping = {x['Id']: (x['Name'], x['ImageUrl']) for x in cheapest['Carriers']}
agents_mapping = {c['Id']:c['ImageUrl'] for c in cheapest['Agents']}
details = {c['Id']: c for c in cheapest['Legs']}

In [147]:
def process_data(c):
    inbound = details[c['InboundLegId']]
    outbound = details[c['OutboundLegId']]

    inbound['date'] = datetime.strptime(inbound['Departure'],'%Y-%m-%dT%H:%M:%S').date()
    outbound['date'] = datetime.strptime(outbound['Departure'],'%Y-%m-%dT%H:%M:%S').date()
    inbound['time'] = datetime.strptime(inbound['Departure'],'%Y-%m-%dT%H:%M:%S').time()
    outbound['time'] = datetime.strptime(outbound['Departure'],'%Y-%m-%dT%H:%M:%S').time()

    inbound['arrival_date'] = datetime.strptime(inbound['Arrival'],'%Y-%m-%dT%H:%M:%S').date()
    outbound['arrival_date'] = datetime.strptime(outbound['Arrival'],'%Y-%m-%dT%H:%M:%S').date()
    inbound['arrival_time'] = datetime.strptime(inbound['Arrival'],'%Y-%m-%dT%H:%M:%S').time()
    outbound['arrival_time'] = datetime.strptime(outbound['Arrival'],'%Y-%m-%dT%H:%M:%S').time()

    print(inbound['Carriers'])
    
    try:
        inbound['Carriers'] = [carriers_mapping[x] for x in inbound['Carriers']]
    except:
        pass
    
    try:
        outbound['Carriers'] = [carriers_mapping[x] for x in outbound['Carriers']]
    except:
        pass




    c['InboundDetails'] = del_keys(inbound)
    c['OutboundDetails'] = del_keys(outbound)

    del c['OutboundLegId']
    del c['InboundLegId']
    del c['BookingDetailsLink']
    for x in c['PricingOptions']:
        x['Agents'] = [agents_mapping[p] for p in x['Agents']]
    return c

In [149]:
process_data(cheapest['Itineraries'][5])

[('LOT', 'http://s1.apideeplink.com/images/airlines/LO.png')]


{'InboundDetails': {'Carriers': [('LOT',
    'http://s1.apideeplink.com/images/airlines/LO.png')],
  'Duration': 120,
  'arrival_date': datetime.date(2017, 3, 25),
  'arrival_time': datetime.time(21, 55),
  'date': datetime.date(2017, 3, 25),
  'time': datetime.time(19, 55)},
 'OutboundDetails': {'Carriers': [('LOT',
    'http://s1.apideeplink.com/images/airlines/LO.png')],
  'Duration': 120,
  'arrival_date': datetime.date(2017, 3, 20),
  'arrival_time': datetime.time(19, 10),
  'date': datetime.date(2017, 3, 20),
  'time': datetime.time(17, 10)},
 'PricingOptions': [{'Agents': ['http://s1.apideeplink.com/images/websites/gtbf.png'],
   'DeeplinkUrl': 'http://partners.api.skyscanner.net/apiservices/deeplink/v2?_cje=glpEOc06Ufm4YL6UUHf3kEolgpMnGr4wmrQheXcA1hhUYcFGUdNQsnXBLcaYkZLR&url=http%3a%2f%2fwww.apideeplink.com%2ftransport_deeplink%2f4.0%2fCH%2fen-gb%2fCHF%2fgtbf%2f2%2f17648.18563.2017-03-20%2c18563.17648.2017-03-25%2fair%2ftrava%2fflights%3fitinerary%3dflight%7c-32093%7c419%7c1764

In [206]:
import json

categories_yelp = requests.get('https://www.yelp.com/developers/documentation/v2/all_category_list/categories.json').json()

from collections import defaultdict
cats = defaultdict(list)
for c in categories_yelp:
    for p in c['parents']:
        cats[p].append({'alias': c['alias'], 'title': c['title']})

root_cats = [c for c in categories_yelp if  not c['parents']]

all_cats = {c['alias']: {'children': [], 'title': c['title']} for c in categories_yelp}

tree = {c['alias']: {'children': all_cats[c['alias']]['children'], 'title': c['title']} for c in root_cats}


for c in categories_yelp:
    for p in c['parents']:
        all_cats[p]['children'].append({c['alias']: all_cats[c['alias']]})

In [261]:
tree.keys()

dict_keys(['food', 'arts', 'financialservices', 'education', 'active', 'shopping', 'eventservices', 'localservices', 'bicycles', 'auto', 'massmedia', 'health', 'restaurants', 'professional', 'hotelstravel', 'publicservicesgovt', 'beautysvc', 'nightlife', 'religiousorgs', 'pets', 'homeservices', 'localflavor'])

In [265]:
from requests_oauthlib import OAuth1

In [271]:
from config import yelp_consumer_key, yelp_consumer_secret, yelp_token, yelp_token_secret

In [272]:
auth = OAuth1(yelp_consumer_key, yelp_consumer_secret, yelp_token,
              yelp_token_secret)

In [281]:
r = requests.get(
    'https://api.yelp.com/v2/search?location={}&category_filter={}'.format(
        'Zurych', 'active'),
    auth=auth)

In [298]:
d = r.json()

In [299]:
keys = {
    'categories', 'display_phone', 'image_url', 'location',
    'rating_img_url_small', 'review_count'
}


def del_keys(x):
    copy = deepcopy(x)
    for key in copy.keys():
        if key not in keys:
            del x[key]
    copy = deepcopy(x)
    for key in copy['location'].keys():
        if key != 'coordinate':
            del x['location'][key]
    return x


return [del_keys(x) for x in d['businesses']]